Imports

In [11]:
from influxdb_client_3 import InfluxDBClient3
import pandas as pd  
import pyarrow.parquet as pq
from pathlib import Path

In this part we perform the Data Backup from influxDB. As a result, sparced dataset with raw data stored in folder 'influxDB Backup' in root folder of this repository. File format: parquet. 


In [12]:
LIMIT = 1_000_000

client = InfluxDBClient3(
    token="apiv3_9EL5Mq-kpLIFd1y3dc9ow0JWWnGPMLM9h6lcFQxQt02FZ-SZis7eJYmj27-E8EKlA_3MTUGa_QBd0holy94oEQ",
    host="http://10.65.1.77:10102",
    database="spp1-main"
)

TABLE_NAME = "saw"
OUTPUT_FILE = "influxDB Backup/saw.backup.parquet"

offset = 0
all_frames = [] # List to store each chunk in memory

print(f"Starting export of '{TABLE_NAME}'...")

while True:
    print(f"Fetching rows {offset} to {offset + LIMIT}...", end="\r")

    # Order by time is critical to ensure pages don't shuffle or overlap
    query = f"""
        SELECT * FROM "{TABLE_NAME}" 
        ORDER BY time ASC
        LIMIT {LIMIT} OFFSET {offset}
    """

    try:
        # Fetch the chunk
        df_chunk = client.query(query=query, mode="pandas")

        # If chunk is empty, we are done
        if df_chunk.empty:
            print(f"\nNo more data found after offset {offset}.")
            break
        
        # Append to our in-memory list
        all_frames.append(df_chunk)

        # Check if this was the last page (less rows than limit)
        if len(df_chunk) < LIMIT:
            print(f"\nReached end of dataset at offset {offset + len(df_chunk)}.")
            break

        # Prepare for next iteration
        offset += LIMIT

    except Exception as e:
        print(f"\nCRITICAL ERROR at offset {offset}: {e}")
        # Depending on importance, you might want to 'break' or 'raise' here
        break

# --- Final Consolidation ---
if all_frames:
    print("Concatenating data...")
    final_df = pd.concat(all_frames, ignore_index=True)

    print(f"Saving {len(final_df)} rows to {OUTPUT_FILE}...")
    final_df.to_parquet(OUTPUT_FILE, index=False)
    print("Export successful.")
else:
    print("No data was retrieved.")

Starting export of 'saw'...
Fetching rows 0 to 1000000...
CRITICAL ERROR at offset 0: Error while executing query: Flight returned unavailable error, with message: failed to connect to all addresses; last error: UNKNOWN: ipv4:10.65.1.77:10102: tcp handshaker shutdown. gRPC client debug context: UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNKNOWN: ipv4:10.65.1.77:10102: tcp handshaker shutdown", grpc_status:14, created_time:"2026-01-04T13:03:48.451546+01:00"}. Client context: IOError: Server never sent a data message. Detail: Internal
No data was retrieved.


Parquet to csv

In [19]:
in_path = Path("influxDB Backup/saw.backup.parquet")
out_path = in_path.with_suffix(".csv")

df = pq.read_table(in_path).to_pandas()
df.to_csv(out_path, index=False)

print("Saved to:", out_path.resolve())
print("Exists:", out_path.exists(), "Size:", out_path.stat().st_size if out_path.exists() else None)


Saved to: /Users/vvsmbk/Desktop/all/3 semester classes/Smart Production and Prototyping/Practical Part/gitProject/projectRaphael/influxDB Backup/saw.backup.csv
Exists: True Size: 7143765


"Compression" of the raw csv file: after that step, all the waterflow presented data reorganized in wide format

In [21]:
df = pd.read_csv("influxDB Backup/saw.backup.csv")

df["time"] = pd.to_datetime(df["time"])
df["time_1s"] = df["time"].dt.floor("s")   # .dt.round("s") for rounding

value_cols = [c for c in df.columns if c not in ("time", "time_1s")]

out = (
    df.sort_values("time")
      .groupby("time_1s")[value_cols]
      .last()                      # last not-NaN for each column
      .reset_index()
      .rename(columns={"time_1s": "time"})
)

out.to_csv("influxDB Backup/saw_wide.csv", index=False)
